In [2]:
from influxdb_client_3 import InfluxDBClient3
import pyarrow as pa
import duckdb

# Connect to InfluxDB 3
client = InfluxDBClient3(
    host="http://localhost:9000",
    token="apiv3_wfr_admin_token_change_in_production",
    database="WFR25"
)

In [29]:
# Check InfluxDB connection
try:
    # Simple query to test connection
    test_query = client.query("SELECT 1 as test")
    print("Connection to InfluxDB successful!")
    print("Test query result:", test_query)
except Exception as e:
    print(f"Connection failed: {e}")

Connection to InfluxDB successful!
Test query result: pyarrow.Table
test: int64 not null
----
test: [[1]]


In [30]:
# Query to find the highest historical value of signal 'INV_DC_Bus_Current'
max_value_table = client.query("""
    SELECT MAX("sensorReading") as max_value
    FROM "iox"."WFR25"
    WHERE "signalName" = 'INV_DC_Bus_Current'
""")

print("Max value query result:")
print(max_value_table)

Max value query result:
pyarrow.Table
max_value: double
----
max_value: [[194]]


In [20]:
# Query to get the latest 10 entries from WFR25
latest_entries = client.query("""
    SELECT time, "sensorReading", "signalName"
    FROM "iox"."WFR25"
    ORDER BY time DESC
    LIMIT 10
""")

print("Latest 10 entries:")
print(latest_entries)

Latest 10 entries:
pyarrow.Table
time: timestamp[ns] not null
sensorReading: double
signalName: string
----
time: [[2025-09-30 00:46:52.104000000,2025-09-30 00:46:52.104000000,2025-09-30 00:46:52.104000000,2025-09-30 00:46:52.104000000,2025-09-30 00:46:52.104000000,2025-09-30 00:46:52.104000000,2025-09-30 00:46:52.104000000,2025-09-30 00:46:52.094000128,2025-09-30 00:46:52.094000128,2025-09-30 00:46:52.094000128]]
sensorReading: [[0,0,0,0,0,0,0,0,0,0]]
signalName: [["VCU_INV_Torque_Command","VCU_INV_Direction_Command","VCU_INV_Torque_Limit_Command","VCU_INV_Speed_Mode_Enable","VCU_INV_Inverter_Enable","VCU_INV_Speed_Command","VCU_INV_Inverter_Discharge","VCU_INV_Speed_Command","VCU_INV_Inverter_Discharge","VCU_INV_Speed_Mode_Enable"]]


In [11]:
# Query to list available tables in the database
tables = client.query("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'iox'
""")

print("Available tables in iox schema:")
print(tables)

Available tables in iox schema:
pyarrow.Table
table_name: string not null
----
table_name: [["WFR25"]]


In [22]:
# Query to list all tables in the database
all_tables = client.query("""
    SELECT table_schema, table_name
    FROM information_schema.tables
""")

print("All tables:")
print(all_tables)

All tables:
pyarrow.Table
table_schema: string not null
table_name: string not null
----
table_schema: [["iox","system","system","system","system",...,"information_schema","information_schema","information_schema","information_schema","information_schema"]]
table_name: [["WFR25","distinct_caches","influxdb_schema","last_caches","parquet_files",...,"columns","df_settings","schemata","routines","parameters"]]


In [16]:
# Check the schema of the WFR25 table
schema_check = client.query("""
    SELECT *
    FROM "iox"."WFR25"
    LIMIT 1
""")

print("Table schema:")
print(schema_check.schema)
print("Sample data:")
print(schema_check)

Table schema:
canId: string
  -- field metadata --
  iox::column::type: 'iox::column_type::tag'
messageName: string
  -- field metadata --
  iox::column::type: 'iox::column_type::tag'
sensorReading: double
  -- field metadata --
  iox::column::type: 'iox::column_type::field::float'
signalName: string
  -- field metadata --
  iox::column::type: 'iox::column_type::tag'
time: timestamp[ns] not null
  -- field metadata --
  iox::column::type: 'iox::column_type::timestamp'
Sample data:
pyarrow.Table
canId: string
messageName: string
sensorReading: double
signalName: string
time: timestamp[ns] not null
----
canId: [["2014"]]
messageName: [["VCU_Front_Sensors_1"]]
sensorReading: [[0.48]]
signalName: [["A1"]]
time: [[2025-09-28 22:19:10.292999936]]


In [32]:
# Print all signal names as a list
signal_list = all_signals['signalName'].to_pylist()
print("All signal names:")
for signal in signal_list:
    print(signal)

All signal names:
A1
A10
A11
A12
A13
A14
A15
A16
A2
A3
A4
A5
A6
A7
A8
A9
AMSRelay
Accel_X
Accel_Y
Accel_Z
BMS_Max_Charge_Current
BMS_Max_Discharge_Current
BSPDRelay
Brake_Percent
Control
Fault
Front_Left_Ticker
Front_Right_Ticker
Gyro_X
Gyro_Y
Gyro_Z
HSD1
HSD2
HSD3
HSD4
HSD5
HSD6
HSD7
HSD8
HV_Active
IMDRelay
INV_Analog_Input_1
INV_Analog_Input_2
INV_Analog_Input_3
INV_Analog_Input_4
INV_Analog_Input_5
INV_Analog_Input_6
INV_BMS_Active
INV_BMS_Torque_Limiting
INV_Burst_Model_Mode
INV_Commanded_Torque
INV_Control_Board_Temp
INV_Coolant_Temp
INV_DC_Bus_Current
INV_DC_Bus_Voltage
INV_DateCode_MMDD
INV_DateCode_YYYY
INV_Delta_Resolver_Filtered
INV_Digital_Input_1
INV_Digital_Input_2
INV_Digital_Input_3
INV_Digital_Input_4
INV_Digital_Input_5
INV_Digital_Input_6
INV_Digital_Input_7
INV_Digital_Input_8
INV_Direction_Command
INV_Electrical_Output_Frequency
INV_Fast_DC_Bus_Voltage
INV_Fast_Motor_Speed
INV_Fast_Torque_Command
INV_Fast_Torque_Feedback
INV_Flux_Weakening_Output
INV_Gate_Driver_Boa

In [34]:
# Query to find rolling 5-second max for M4_Cell10_Voltage
rolling_max_5s = client.query("""
    SELECT 
        (EXTRACT(epoch from time) / 5)::bigint * 5 as bucket_epoch,
        MAX("sensorReading") as max_voltage
    FROM "iox"."WFR25"
    WHERE "signalName" = 'M4_Cell10_Voltage'
    GROUP BY bucket_epoch
    ORDER BY bucket_epoch
""")

print("Rolling 5-second max for M4_Cell10_Voltage:")
print(rolling_max_5s)

Rolling 5-second max for M4_Cell10_Voltage:
pyarrow.Table
bucket_epoch: int64
max_voltage: double
----
bucket_epoch: [[1759097945,1759097950,1759097955,1759097960,1759097965,...,1759193190,1759193195,1759193200,1759193205,1759193210]]
max_voltage: [[4.0325,4.0325,4.0324,4.0325,4.0324,...,3.3091000000000004,3.309,3.3088,3.3091000000000004,3.3091000000000004]]


In [3]:
# Fetch data for M4_Cell10_Voltage
voltage_data = client.query("""
    SELECT time, "sensorReading"
    FROM "iox"."WFR25"
    WHERE "signalName" = 'M4_Cell10_Voltage'
    ORDER BY time
""")

# Convert to pandas for analysis
import pandas as pd
df = voltage_data.to_pandas()
df.set_index('time', inplace=True)

# Calculate rolling median with 5-second window
rolling_median = df['sensorReading'].rolling('5s').median()

# Find the maximum median and its time
max_median = rolling_median.max()
max_time = rolling_median.idxmax()

# The period is from max_time - 5s to max_time
start_time = max_time - pd.Timedelta(seconds=5)
end_time = max_time

print(f"Highest median: {max_median}")
print(f"Period start: {start_time}")
print(f"Period end: {end_time}")

Highest median: 4.0325
Period start: 2025-09-28 22:19:04.440999936
Period end: 2025-09-28 22:19:09.440999936
